# NER using GPT-3.5

### Project name: Honos
Date: 24th May 2024

Author: Milindi Kodikara

Supervisor: Professor Karin Verspoor

Guidelines before running the notebook:
1. Add the correct filename paths for train data, test data, dev data

Libraries used: 
- pd
- numpy

In [ ]:
import pandas as pd
import re

import json


### Step 1: Load and pre-process data and prompt library 


#### Step 1.1: Load datasets

In [ ]:
# train_text.tsv
# pmid\tfilename\ttext

# TODO: Replace filepath for related data file
data = pd.read_csv("./genovardis_train_dev/train_text.tsv", sep='\t', header=0)

data.head(5)

In [ ]:
len(data)

In [ ]:
# TODO: remove this after testing
# data = data.head(2)

# data

In [ ]:
# clean up text by removing the appended pmid and title abstract tags at the start of each section

pattern = '(?:[\d]{1,10}\|t\|)(?P<title>[\w\W]+)(?:\\n[\d]{1,20}\|a\|)(?P<abstract>[\w\W]+)'

def clean_text(text):
    matches = re.search(pattern, text)
    reformatted_text = f'{matches.group("title")}\n{matches.group("abstract")}'
    return reformatted_text

data['text'] = [clean_text(text) for text in data['text']]


data

In [ ]:
len(data)


#### Step 1.2: Load prompt library

In [ ]:
prompt_library = pd.read_json('prompts.json')

prompt_library


#### Step 1.3: Create data+prompt dataset

In [ ]:
# TODO: Buff up the prompts 
# pmid prompt_id embedded_prompt
def embed_data_in_prompts(row_data):
    prompts = []
    pmid = row_data['pmid']
    data_text = row_data['text']
    
    for index, row_prompt in prompt_library.iterrows():
        instruction = row_prompt['instruction']
        prompt_text = row_prompt['text'].format(data_text)
        # TODO: Figure out the new line characters 
        concatenated_prompt = '{}\n"{}"'.format(instruction, prompt_text)
        
        prompt = {'prompt_id': row_prompt['prompt_id'], 'prompt': concatenated_prompt}
        prompts.append(prompt)
    
    return {'pmid': pmid, 'prompts': prompts}


In [ ]:

embedded_prompt_data_list = [embed_data_in_prompts(row_data) for index, row_data in data.iterrows()]

In [ ]:
embedded_prompt_data_list[0]


### Step 2: Setting up GPT-3.5

In [ ]:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?tabs=command-line%2Cpython-new&pivots=programming-language-python

### Step 3: Post-processing


### Step 4: Evaluation

*Skip this part for evaluation dataset as there is no gold standard data to compare against.*

In [ ]:
# train_annotations.tsv
# pmid\tfilename\tmark\tlabel\toffset1\toffset2\tspan

def evaluate(eval_filepath=None, results=None):
    if eval_filepath is not None:
        
        gold_standard_annotations = pd.read_csv(eval_filepath, sep='\t', header=0)
        print(gold_standard_annotations.sample(5))

In [ ]:
evaluate("./genovardis_train_dev/train_annotation.tsv")


### Step 5: Saving output

`.tsv` file containing the annotations in the following format: 

`pmid   filename   label   offset1   offset2   span`.

